# Introduction

In this project, we will be making a database of baseball statistics from Retrosheet with the following files to assist us:

- game_log.csv: provides the details of each game captured
- game_log_fields.txt: explains what each field means in game_log.csv
- appearance_type.csv: provides explanations on the codes for the appearance of the player (whether they were the 1st batter, winning pitcher, etc.) that appear in the game_log file
- park_codes.csv: provides explanations on the park codes used in the game_log file
- person_codes.csv: provides explanations on the person codes used in the game_log file
- team_codes.csv: provides explanations on the team codes used in the game_log file

Now, let's jump right in by reading in our datasets.

In [1]:
# Import our needed libraries
import pandas as pd
import sqlite3

# Since the game_log.csv file is very big, we'll run the following code to limit the number of records we'll display
pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth', 5000)

# Read each of the csv's into a pandas dataset
game_log = pd.read_csv("game_log.csv",low_memory=False)
appearance_type = pd.read_csv("appearance_type.csv")
park_codes = pd.read_csv("park_codes.csv")
person_codes = pd.read_csv("person_codes.csv")
team_codes = pd.read_csv("team_codes.csv")

Now that we have our datasets written into pandas dataframes, let's create some helper functions for running our queries.

In [2]:
# create the run_query function using mlb.db as the database
def run_query(q):
  with sqlite3.connect('mlb.db') as conn:
    return pd.read_sql(q, conn)

# create the run_command function
def run_command(c):
  with sqlite3.connect('mlb.db') as conn:
    # enforce foreign key restraints. sqlite does not enforce foreign key restraints by default
    conn.execute('PRAGMA foreign_keys = ON;')
    conn.isolation_level = None
    conn.execute(c)
    
# create the show_tables function
def show_tables():
    # define the query to get all of the views and tables
    q = """SELECT
             name,
             type
           FROM sqlite_master
           WHERE type IN ("table","view");"""
    # use the run_query function to run the query
    return run_query(q)

Now that we've created our helper functions, let's import our dataframes into sqlite3 db tables.

In [3]:
# create the connection to our mlb.db
conn = sqlite3.connect('mlb.db')

# import the dataframes into the db as tables
game_log.to_sql('game_log',conn,if_exists='replace',index=False)
park_codes.to_sql('park_codes',conn,if_exists='replace',index=False)
person_codes.to_sql('person_codes',conn,if_exists='replace',index=False)
team_codes.to_sql('team_codes',conn,if_exists='replace',index=False)

# Note: appearance_type data will be used for our manually created appearance_type table where we set the primary key
# Hence, we are naming the table here "appearance_type_original"
appearance_type.to_sql('appearance_type_original',conn,if_exists='replace',index=False)

In [4]:
# Show the tables to make sure that we have created all of the tables
show_tables()

,name,type
0,person,table
1,park,table
2,league,table
3,appearance_type,table
4,team,table
5,game_log,table
6,park_codes,table
7,person_codes,table
8,team_codes,table
9,appearance_type_original,table


Now that we have created our tables in our db, let's add a game_id primary key to our game_log table.

In [5]:
# Write a query for adding a primary key column to the game_log table that 
q = """
       ALTER TABLE game_log
       ADD COLUMN game_id TEXT;
    """
# run the query
run_command(q)

In [6]:
# Write a query for checking that the primary key has been added
q = """
       SELECT * FROM game_log
       LIMIT 10;
    """
# run the query
run_query(q)

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info,game_id
0,18710504,0,Thu,CL1,None,1,FW1,None,1,0,2,54.0,D,None,None,None,FOR01,200.0,120.0,000000000,010010000,30.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,None,6.0,1.0,None,-1.0,None,4.0,1.0,1.0,1.0,0.0,0.0,27.0,9.0,0.0,3.0,0.0,0.0,31.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,None,0.0,0.0,None,-1.0,None,3.0,1.0,0.0,0.0,0.0,0.0,27.0,3.0,3.0,1.0,1.0,0.0,boakj901,John Boake,None,None,None,None,None,None,None,None,None,None,paboc101,Charlie Pabor,lennb101,Bill Lennon,mathb101,Bobby Mathews,prata101,Al Pratt,None,None,None,None,prata101,Al Pratt,mathb101,Bobby Mathews,whitd102,Deacon White,2.0,kimbg101,Gene Kimball,4.0,paboc101,Charlie Pabor,7.0,allia101,Art Allison,8.0,white104,Elmer White,9.0,prata101,Al Pratt,1.0,sutte101,Ezra Sutton,5.0,carlj102,Jim Carleton,3.0,bassj101,John Bass,6.0,selmf101,Frank Sellman,5.0,mathb101,Bobby Mathews,1.0,foraj101,Jim Foran,3.0,goldw101,Wally Goldsmith,6.0,lennb101,Bill Lennon,2.0,caret101,Tom Carey,4.0,mince101,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,None,Y,None
1,18710505,0,Fri,BS1,None,1,WS3,None,1,20,18,54.0,D,None,None,None,WAS01,5000.0,145.0,107000435,640113030,41.0,13.0,1.0,2.0,0.0,13.0,0.0,0.0,0.0,18.0,None,5.0,3.0,None,-1.0,None,12.0,1.0,6.0,6.0,1.0,0.0,27.0,13.0,10.0,1.0,2.0,0.0,49.0,14.0,2.0,0.0,0.0,11.0,0.0,0.0,0.0,10.0,None,2.0,1.0,None,-1.0,None,14.0,1.0,7.0,7.0,0.0,0.0,27.0,20.0,10.0,2.0,3.0,0.0,dobsh901,Henry Dobson,None,None,None,None,None,None,None,None,None,None,wrigh101,Harry Wright,younn801,Nick Young,spala101,Al Spalding,braia102,Asa Brainard,None,None,None,None,spala101,Al Spaldin

Now that we have the game_id column, let's write a query updating the game_id value to be the game_id that Retrosheet uses:
- First three characters are the home team id
- The next four characters are the year the game was played
- The next two characters are the month
- The next two characters are the day
- The last character denotes a single game (0) or first game (1)/second game (2) if it is a double header (two games played in one day)

In [7]:
# Write a query for updating the game_id column to be the above value:
q = """
       UPDATE game_log
         SET game_id = h_name || date || number_of_game;
    """
# run the query
run_command(q)

In [8]:
# Write a query for checking that the primary key value has been updated correctly
q = """
       SELECT 
         h_name, 
         date, 
         number_of_game,
         h_name || date || number_of_game AS manual_key_to_compare,
         game_id
       FROM game_log
       LIMIT 10;
    """
# run the query
run_query(q)

,h_name,date,number_of_game,manual_key_to_compare,game_id
0,FW1,18710504,0,FW1187105040,FW1187105040
1,WS3,18710505,0,WS3187105050,WS3187105050
2,RC1,18710506,0,RC1187105060,RC1187105060
3,CH1,18710508,0,CH1187105080,CH1187105080
4,TRO,18710509,0,TRO187105090,TRO187105090
5,CL1,18710511,0,CL1187105110,CL1187105110
6,CL1,18710513,0,CL1187105130,CL1187105130
7,FW1,18710513,0,FW1187105130,FW1187105130
8,FW1,18710515,0,FW1187105150,FW1187105150
9,BS1,18710516,0,BS1187105160,BS1187105160


Now that we've created our primary key in the game_log table, let's look for ways to normalize our database. 

After examining the game log and our other tables, some opportunities for normalization are below:
- Reduce repetition in columns
 - Multiple players in one row
 - Multiple umpires in one row
 - For every visitor column, there is a home column
- Non-primary key columns should be attributes of the primary key
 - Remove player names and player ids since the names and player ids are captured in the person_codes table. We would just need to bring the game id to that table
 - The same above can be applied to the umpires
- Redundant data - In the park codes table, the 'start' and 'end' columns are redundant as we can just look at the minimum of the date a game was played there for the start and the max for the end.

With those normalization ideas in mind, let's look at creating our database schema using the tool https://www.dbdesigner.net.

Below is the data model we created using the above tool:

![Alt text](data_model.png "Data Model")

For the sake of following along with the Dataquest exercise, we're going to use their schema below:

![Alt text](data_model_to_be_used_from_dataquest.png "Data Model from Dataquest")

# Comparing my data model to Dataquest's data model

Below are the similarities and differences I see with the model I created and Dataquest's model:

- Similarities (note: used the Dataquest table names)
  - team_appearance table
  - team table
  - person table
  - appearance type table
  - park table
  - game table
  - person_appearance table general idea
  
- Major Differences
  - league table (looks like they decided to create a separate leagues table)
  - person_appearance table - the use of an integer, auto-incremented primary key rather than a compound primary key like I had. Though I think both ways will work, I like Dataquest's version more as I think a new, single primary key is easier to see/read than a compound primary key

# Creating our database from our model

Now that we have our data model, we'll start creating the tables in our database. First, we want to create the tables that do not have any foreign keys, which are the following table:
- person
- park
- league
- appearance_type

Below, we'll create the person table using data from our person_codes table.

In [9]:
# Create the person table within our database
q = """
       CREATE TABLE IF NOT EXISTS person (
         person_id TEXT PRIMARY KEY,
         first_name TEXT,
         last_name TEXT
       );
    """
# run the query
run_command(q)

# insert the person values into our newly created table
q2 = """
       INSERT OR IGNORE INTO person
         SELECT
           id,
           first,
           last
         FROM person_codes
       ;
    """
# run the query
run_command(q2)

# select the first 10 rows of our table
q3 = """
       SELECT * 
       FROM person 
       LIMIT 10
       ;
    """
# run the query
run_query(q3)

,person_id,first_name,last_name
0,aardd001,David,Aardsma
1,aaroh101,Hank,Aaron
2,aarot101,Tommie,Aaron
3,aased001,Don,Aase
4,abada001,Andy,Abad
5,abadf001,Fernando,Abad
6,abadj101,John,Abadie
7,abbae101,Ed,Abbaticchio
8,abbeb101,Bert,Abbey
9,abbec101,Charlie,Abbey


Next, we'll create the park table from our park_codes table.

In [10]:
# Create the park table within our database
q = """
       CREATE TABLE IF NOT EXISTS park (
         park_id TEXT PRIMARY KEY,
         name TEXT,
         nickname TEXT,
         city TEXT,
         state TEXT,
         notes TEXT
       );
    """
# run the query
run_command(q)

# insert the park values into our newly created table
q2 = """
       INSERT OR IGNORE INTO park
         SELECT
           park_id,
           name,
           aka,
           city,
           state,
           notes
         FROM park_codes
       ;
    """
# run the query
run_command(q2)

# select the first 10 rows of our table
q3 = """
       SELECT * 
       FROM park
       LIMIT 10
       ;
    """
# run the query
run_query(q3)

,park_id,name,nickname,city,state,notes
0,ALB01,Riverside Park,None,Albany,NY,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,None,Altoona,PA,None
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,None
3,ARL01,Arlington Stadium,None,Arlington,TX,None
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,None
5,ATL01,Atlanta-Fulton County Stadium,None,Atlanta,GA,None
6,ATL02,Turner Field,None,Atlanta,GA,None
7,ATL03,Suntrust Park,None,Atlanta,GA,None
8,BAL01,Madison Avenue Grounds,None,Baltimore,MD,WS3
9,BAL02,Newington Park,None,Baltimore,MD,BL1:1872-74; BL4:1873; BL2: 1882


Now, we'll create the league table. Unfortunately, we do not have a league table already, but luckily, we only have six different leagues to insert our table. As a result, we'll insert the league names manually based on the distinct values for leagues in the game_log table.

In [11]:
# pull the distinct leagues
q = """
       SELECT 
         DISTINCT league
       FROM team_codes
       LIMIT 10
       ;
    """
# run the query
run_query(q)

,league
0,UA
1,NL
2,PL
3,None
4,AA
5,AL
6,FL


Now that we have the list of distinct leagues, from doing some searches on the web, here's what we have for the name of each league:
- UA: Union Association (https://en.wikipedia.org/wiki/Union_Association)
- NL: National League (https://en.wikipedia.org/wiki/National_League)
- PL: Players' League (https://en.wikipedia.org/wiki/Players%27_League)
- AA: American Association (https://en.wikipedia.org/wiki/American_Association_(19th_century))
- AL: American League (https://en.wikipedia.org/wiki/American_League)
- FL: Federal League (https://en.wikipedia.org/wiki/Federal_League)

From there, we can now create our league table and insert the above data.

In [12]:
# Create the league table within our database
q = """
       CREATE TABLE IF NOT EXISTS league (
         league_id TEXT PRIMARY KEY,
         name TEXT
       );
    """
# run the query
run_command(q)

# insert the league values into our newly created table
q2 = """
       INSERT OR IGNORE INTO league
       VALUES
       ('UA','Union Association'),
       ('NL','National League'),
       ('PL','Players'' League'),
       ('AA','American Association'),
       ('AL','American League'),
       ('FL','Federal League')
       ;
    """
# run the query
run_command(q2)

# select the first 10 rows of our table
q3 = """
       SELECT * 
       FROM league
       LIMIT 10
       ;
    """
# run the query
run_query(q3)

,league_id,name
0,UA,Union Association
1,NL,National League
2,PL,Players' League
3,AA,American Association
4,AL,American League
5,FL,Federal League


Finally, we just need the appearance_type table. We actually created an appearance_type table from the appearance_type dataframe that had the data from appearance_type.csv; however, in order to set the primary key of the table, we need to recreate the table.

In [13]:
# Create the appearance table within our database
q = """
       CREATE TABLE IF NOT EXISTS appearance_type (
         appearance_type_id TEXT PRIMARY KEY,
         name TEXT,
         category TEXT
       );
    """
# run the query
run_command(q)

# insert the appearance values into our newly created table
q2 = """
       INSERT OR IGNORE INTO appearance_type
         SELECT *
         FROM appearance_type_original
       ;
    """
# run the query
run_command(q2)

# select the first 10 rows of our table
q3 = """
       SELECT * 
       FROM appearance_type
       ;
    """
# run the query
run_query(q3)

,appearance_type_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense
5,O6,Batter 6,offense
6,O7,Batter 7,offense
7,O8,Batter 8,offense
8,O9,Batter 9,offense
9,D1,Pitcher,defense


With our base tables created, now we can create the following tables that have a foreign key restraint based on one of our previously created tables:
- team
- game

We'll create the team table first.

In [14]:
# Create the team table within our database
q = """
       CREATE TABLE IF NOT EXISTS team (
         team_id TEXT PRIMARY KEY,
         league_id TEXT,
         city TEXT,
         nickname TEXT,
         franch_id TEXT,
         FOREIGN KEY (league_id) REFERENCES league(league_id)
       );
    """
# run the query
run_command(q)

# insert the team values into our newly created table
q2 = """
       INSERT OR IGNORE INTO team
         SELECT 
           team_id,
           league,
           city,
           nickname,
           franch_id
         FROM team_codes
       ;
    """
# run the query
run_command(q2)

# select the first 10 rows of our table
q3 = """
       SELECT * 
       FROM team
       LIMIT 10
       ;
    """
# run the query
run_query(q3)

,team_id,league_id,city,nickname,franch_id
0,ALT,UA,Altoona,Mountain Cities,ALT
1,ARI,NL,Arizona,Diamondbacks,ARI
2,BFN,NL,Buffalo,Bisons,BFN
3,BFP,PL,Buffalo,Bisons,BFP
4,BL1,None,Baltimore,Canaries,BL1
5,BL2,AA,Baltimore,Orioles,BL2
6,BLN,NL,Baltimore,Orioles,BL2
7,BL4,None,Baltimore,Marylands,BL4
8,BLA,AL,Baltimore,Orioles,BLA
9,NYA,AL,New York,Yankees,BLA


Now, let's create the game table.

In [38]:
# Create the game table within our database
q = """
       CREATE TABLE IF NOT EXISTS game (
         game_id TEXT PRIMARY KEY,
         date INTEGER,
         number_of_game INTEGER,
         park_id TEXT,
         length_outs INTEGER,
         day INTEGER,
         completion TEXT,
         forfeit TEXT,
         protest TEXT,
         attendance INTEGER,
         length_minutes INTEGER,
         additional_info TEXT,
         acquisition_info TEXT,
         FOREIGN KEY (park_id) REFERENCES park(park_id)
       );
    """
# run the query
run_command(q)

# insert the game values into our newly created table
q2 = """
       INSERT OR IGNORE INTO game
         SELECT 
           game_id,
           date,
           number_of_game,
           park_id,
           length_outs,
           day_night,
           completion,
           forefeit,
           protest,
           attendance,
           length_minutes,
           additional_info,
           acquisition_info
         FROM game_log
       ;
    """
# run the query
run_command(q2)

# select the first 10 rows of our table
q3 = """
       SELECT * 
       FROM game
       LIMIT 10
       ;
    """
# run the query
run_query(q3)

,game_id,date,number_of_game,park_id,length_outs,day,completion,forfeit,protest,attendance,length_minutes,additional_info,acquisition_info
0,FW1187105040,18710504,0,FOR01,54,D,None,None,None,200.0,120.0,None,Y
1,WS3187105050,18710505,0,WAS01,54,D,None,None,None,5000.0,145.0,HTBF,Y
2,RC1187105060,18710506,0,RCK01,54,D,None,None,None,1000.0,140.0,None,Y
3,CH1187105080,18710508,0,CHI01,54,D,None,None,None,5000.0,150.0,None,Y
4,TRO187105090,18710509,0,TRO01,54,D,None,None,None,3250.0,145.0,HTBF,Y
5,CL1187105110,18710511,0,CLE01,48,D,None,V,None,2500.0,120.0,None,Y
6,CL1187105130,18710513,0,CIN01,54,D,None,None,None,1200.0,150.0,None,Y
7,FW1187105130,18710513,0,FOR01,54,D,None,None,None,1500.0,105.0,None,Y
8,FW1187105150,18710515,0,FOR01,54,D,None,None,None,NaN,140.0,None,Y
9,BS1187105160,18710516,0,BOS01,54,D,None,None,None,2500.0,NaN,HTBF,Y


With our game and team tables created, we can now create the team_appearance table. In order to make our lives easier when creating this table, we'll use the structure of our game_log table as a starting point.

In [44]:
# show the create statement for the game_log table
q = """
       SELECT sql 
       FROM sqlite_master
       WHERE name = "game_log" AND type = "table"
       ;
    """
# run the query
run_query(q)

sql
0  CREATE TABLE "game_log" (\n"date" INTEGER,\n  "number_of_game" INTEGER,\n  "day_of_week" TEXT,\n  "v_name" TEXT,\n  "v_league" TEXT,\n  "v_game_number" INTEGER,\n  "h_name" TEXT,\n  "h_league" TEXT,\n  "h_game_number" INTEGER,\n  "v_score" INTEGER,\n  "h_score" INTEGER,\n  "length_outs" REAL,\n  "day_night" TEXT,\n  "completion" TEXT,\n  "forefeit" TEXT,\n  "protest" TEXT,\n  "park_id" TEXT,\n  "attendance" REAL,\n  "length_minutes" REAL,\n  "v_line_score" TEXT,\n  "h_line_score" TEXT,\n  "v_at_bats" REAL,\n  "v_hits" REAL,\n  "v_doubles" REAL,\n  "v_triples" REAL,\n  "v_homeruns" REAL,\n  "v_rbi" REAL,\n  "v_sacrifice_hits" REAL,\n  "v_sacrifice_flies" REAL,\n  "v_hit_by_pitch" REAL,\n  "v_walks" REAL,\n  "v_intentional_walks" REAL,\n  "v_strikeouts" REAL,\n  "v_stolen_bases" REAL,\n  "v_caught_stealing" REAL,\n  "v_grounded_into_double" REAL,\n  "v_first_catcher_interference" REAL,\n  "v_left_on_base" REAL,\n  "v_pitchers_used" REAL,\n  "v_individual_earned_runs" REAL,\n  "v_team_earned_runs" REAL,\n  "v_wild_pitches" REAL,\n  "v_balks" REAL,\n  "v_putouts" REAL,\n  "v_assists" REAL,\n  "v_errors" REAL,\n  "v_passed_balls" REAL,\n  "v_double_plays" REAL,\n  "v_triple_plays" REAL,\n  "h_at_bats" REAL,\n  "h_hits" REAL,\n  "h_doubles" REAL,\n  "h_triples" REAL,\n  "h_homeruns" REAL,\n  "h_rbi" REAL,\n  "h_sacrifice_hits" REAL,\n  "h_sacrifice_flies" REAL,\n  "h_hit_by_pitch" REAL,\n  "h_walks" REAL,\n  "h_intentional_walks" REAL,\n  "h_strikeouts" REAL,\n  "h_stolen_bases" REAL,\n  "h_caught_stealing" REAL,\n  "h_grounded_into_double" REAL,\n  "h_first_catcher_interference" REAL,\n  "h_left_on_base" REAL,\n  "h_pitchers_used" REAL,\n  "h_individual_earned_runs" REAL,\n  "h_team_earned_runs" REAL,\n  "h_wild_pitches" REAL,\n  "h_balks" REAL,\n  "h_putouts" REAL,\n  "h_assists" REAL,\n  "h_errors" REAL,\n  "h_passed_balls" REAL,\n  "h_double_plays" REAL,\n  "h_triple_plays" REAL,\n  "hp_umpire_id" TEXT,\n  "hp_umpire_name" TEXT,\n  "1b_umpire_id" TEXT,\n  "1b_umpire_name" TEXT,\n  "2b_umpire_id" TEXT,\n  "2b_umpire_name" TEXT,\n  "3b_umpire_id" TEXT,\n  "3b_umpire_name" TEXT,\n  "lf_umpire_id" TEXT,\n  "lf_umpire_name" TEXT,\n  "rf_umpire_id" TEXT,\n  "rf_umpire_name" TEXT,\n  "v_manager_id" TEXT,\n  "v_manager_name" TEXT,\n  "h_manager_id" TEXT,\n  "h_manager_name" TEXT,\n  "winning_pitcher_id" TEXT,\n  "winning_pitcher_name" TEXT,\n  "losing_pitcher_id" TEXT,\n  "losing_pitcher_name" TEXT,\n  "saving_pitcher_id" TEXT,\n  "saving_pitcher_name" TEXT,\n  "winning_rbi_batter_id" TEXT,\n  "winning_rbi_batter_id_name" TEXT,\n  "v_starting_pitcher_id" TEXT,\n  "v_starting_pitcher_name" TEXT,\n  "h_starting_pitcher_id" TEXT,\n  "h_starting_pitcher_name" TEXT,\n  "v_player_1_id" TEXT,\n  "v_player_1_name" TEXT,\n  "v_player_1_def_pos" REAL,\n  "v_player_2_id" TEXT,\n  "v_player_2_name" TEXT,\n  "v_player_2_def_pos" REAL,\n  "v_player_3_id" TEXT,\n  "v_player_3_name" TEXT,\n  "v_player_3_def_pos" REAL,\n  "v_player_4_id" TEXT,\n  "v_player_4_name" TEXT,\n  "v_player_4_def_pos" REAL,\n  "v_player_5_id" TEXT,\n  "v_player_5_name" TEXT,\n  "v_player_5_def_pos" REAL,\n  "v_player_6_id" TEXT,\n  "v_player_6_name" TEXT,\n  "v_player_6_def_pos" REAL,\n  "v_player_7_id" TEXT,\n  "v_player_7_name" TEXT,\n  "v_player_7_def_pos" REAL,\n  "v_player_8_id" TEXT,\n  "v_player_8_name" TEXT,\n  "v_player_8_def_pos" REAL,\n  "v_player_9_id" TEXT,\n  "v_player_9_name" TEXT,\n  "v_player_9_def_pos" REAL,\n  "h_player_1_id" TEXT,\n  "h_player_1_name" TEXT,\n  "h_player_1_def_pos" REAL,\n  "h_player_2_id" TEXT,\n  "h_player_2_name" TEXT,\n  "h_player_2_def_pos" REAL,\n  "h_player_3_id" TEXT,\n  "h_player_3_name" TEXT,\n  "h_player_3_def_pos" REAL,\n  "h_player_4_id" TEXT,\n  "h_player_4_name" TEXT,\n  "h_player_4_def_pos" REAL,\n  "h_player_5_id" TEXT,\n  "h_player_5_name" TEXT,\n  "h_player_5_def_pos" REAL,\n  "h_player_6_id" TEXT,\n  "h_player_6_name" TEXT,\n  "h_player_6_def_pos" REAL,\n  "h_player_7_id" TEXT,\n  "h_player_7_name" TEXT,\n  "h_playe

This gives us a starting point at least. I copied and pasted the above into TextEdit to be able to more easily work with this block of text. Below is the final result of our CREATE table statement:

In [48]:
# create the team_appearance table
q = """
       CREATE TABLE IF NOT EXISTS team_appearance
(
team_id TEXT,
game_id TEXT,
home INTEGER,
league_id TEXT,
score INTEGER,
line_score TEXT,
at_bats INTEGER,
hits INTEGER,
doubles INTEGER,
triples INTEGER,
homeruns INTEGER,
rbi INTEGER,
sacrifice_hits INTEGER,
sacrifice_flies INTEGER,
hit_by_pitch INTEGER,
walks INTEGER,
intentional_walks INTEGER,
strikeouts INTEGER,
stolen_bases INTEGER,
caught_stealing INTEGER,
grounded_into_double INTEGER,
first_catcher_interference INTEGER,
left_on_base INTEGER,
pitchers_used INTEGER,
individual_earned_runs INTEGER,
team_earned_runs INTEGER,
wild_pitches INTEGER,
balks INTEGER,
putouts INTEGER,
assists INTEGER,
errors INTEGER,
passed_balls INTEGER,
double_plays INTEGER,
triple_plays INTEGER,
PRIMARY KEY (team_id, game_id)
FOREIGN KEY (team_id) REFERENCES team(team_id),
FOREIGN KEY (game_id) REFERENCES game(game_id),
FOREIGN KEY (league_id) REFERENCES league(league_id)
)
       ;
    """
# run the query
run_command(q)

# check that our table was created successfully
# select the first 10 rows of our table
q3 = """
       SELECT * 
       FROM team_appearance
       LIMIT 10
       ;
    """
# run the query
run_query(q3)

,team_id,game_id,home,league_id,score,line_score,at_bats,hits,doubles,triples,homeruns,rbi,sacrifice_hits,sacrifice_flies,hit_by_pitch,walks,intentional_walks,strikeouts,stolen_bases,caught_stealing,grounded_into_double,first_catcher_interference,left_on_base,pitchers_used,individual_earned_runs,team_earned_runs,wild_pitches,balks,putouts,assists,errors,passed_balls,double_plays,triple_plays


Now that we have created our table, we now need to insert the data from the game_log table. As a note, since the game_log table is flat/de-normalized and has the same columns for both home and visitor whereas our team_appearance table uses the home field as a boolean, we'll need to do a union statement of the home and visitor values.

In [55]:
# Insert the values from the game_log table into our team_appearance table
# insert the game values into our newly created table
q2 = """
       INSERT OR IGNORE INTO team_appearance
         SELECT
        h_name,
        game_id,
        1 AS home,
        h_league,
        h_score,
        h_line_score,
        h_at_bats,
        h_hits,
h_doubles,
h_triples,
h_homeruns,
h_rbi,
h_sacrifice_hits,
h_sacrifice_flies,
h_hit_by_pitch,
h_walks,
h_intentional_walks,
h_strikeouts,
h_stolen_bases,
h_caught_stealing,
h_grounded_into_double,
h_first_catcher_interference,
h_left_on_base,
h_pitchers_used,
h_individual_earned_runs,
h_team_earned_runs,
h_wild_pitches,
h_balks,
h_putouts,
h_assists,
h_errors,
h_passed_balls,
h_double_plays,
h_triple_plays

    FROM game_log

UNION

    SELECT    
        v_name,
        game_id,
        0 AS home,
        v_league,
        v_score,
        v_line_score,
        v_at_bats,
        v_hits,
v_doubles,
v_triples,
v_homeruns,
v_rbi,
v_sacrifice_hits,
v_sacrifice_flies,
v_hit_by_pitch,
v_walks,
v_intentional_walks,
v_strikeouts,
v_stolen_bases,
v_caught_stealing,
v_grounded_into_double,
v_first_catcher_interference,
v_left_on_base,
v_pitchers_used,
v_individual_earned_runs,
v_team_earned_runs,
v_wild_pitches,
v_balks,
v_putouts,
v_assists,
v_errors,
v_passed_balls,
v_double_plays,
v_triple_plays

    from game_log
       ;
    """
# run the query
run_command(q2)

# select the first 10 rows of our table
q3 = """
       SELECT * 
       FROM team_appearance
       LIMIT 10
       ;
    """
# run the query
run_query(q3)

,team_id,game_id,home,league_id,score,line_score,at_bats,hits,doubles,triples,homeruns,rbi,sacrifice_hits,sacrifice_flies,hit_by_pitch,walks,intentional_walks,strikeouts,stolen_bases,caught_stealing,grounded_into_double,first_catcher_interference,left_on_base,pitchers_used,individual_earned_runs,team_earned_runs,wild_pitches,balks,putouts,assists,errors,passed_balls,double_plays,triple_plays
0,ALT,ALT188404300,1,UA,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,ALT,ALT188405020,1,UA,3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,ALT,ALT188405030,1,UA,5,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,ALT,ALT188405050,1,UA,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,ALT,ALT188405100,1,UA,9,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
5,ALT,ALT188405120,1,UA,3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6,ALT,ALT188405140,1,UA,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
7,ALT,ALT188405150,1,UA,7,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
8,ALT,ALT188405160,1,UA,6,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
9,ALT,ALT188405170,1,UA,8,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


With our data inserted, I'm going to check the count of the Select query for game_log within the INSERT compared to the count of records in team_appearance

In [56]:
# Validate the count matches
q3 = """
SELECT COUNT(*)
FROM (
       SELECT
        h_name,
        game_id,
        1 AS home,
        h_league,
        h_score,
        h_line_score,
        h_at_bats,
        h_hits,
h_doubles,
h_triples,
h_homeruns,
h_rbi,
h_sacrifice_hits,
h_sacrifice_flies,
h_hit_by_pitch,
h_walks,
h_intentional_walks,
h_strikeouts,
h_stolen_bases,
h_caught_stealing,
h_grounded_into_double,
h_first_catcher_interference,
h_left_on_base,
h_pitchers_used,
h_individual_earned_runs,
h_team_earned_runs,
h_wild_pitches,
h_balks,
h_putouts,
h_assists,
h_errors,
h_passed_balls,
h_double_plays,
h_triple_plays

    FROM game_log

UNION

    SELECT    
        v_name,
        game_id,
        0 AS home,
        v_league,
        v_score,
        v_line_score,
        v_at_bats,
        v_hits,
v_doubles,
v_triples,
v_homeruns,
v_rbi,
v_sacrifice_hits,
v_sacrifice_flies,
v_hit_by_pitch,
v_walks,
v_intentional_walks,
v_strikeouts,
v_stolen_bases,
v_caught_stealing,
v_grounded_into_double,
v_first_catcher_interference,
v_left_on_base,
v_pitchers_used,
v_individual_earned_runs,
v_team_earned_runs,
v_wild_pitches,
v_balks,
v_putouts,
v_assists,
v_errors,
v_passed_balls,
v_double_plays,
v_triple_plays

    FROM game_log
) 
       ;
    """
# run the query
run_query(q3)

,COUNT(*)
0,343814


In [57]:
# select the first 10 rows of our table
q3 = """
       SELECT COUNT(*)
       FROM team_appearance
       ;
    """
# run the query
run_query(q3)

,COUNT(*)
0,343814


Since both query counts match, we're good to create our last table tracking each person's appearance.

In [58]:
# create the person_appearance table
q = """
       CREATE TABLE IF NOT EXISTS person_appearance
(
appearance_id INTEGER PRIMARY KEY,
person_id TEXT,
team_id TEXT,
game_id TEXT,
appearance_type_id TEXT,
FOREIGN KEY (person_id) REFERENCES person(person_id),
FOREIGN KEY (team_id) REFERENCES team(team_id),
FOREIGN KEY (game_id) REFERENCES game(game_id),
FOREIGN KEY (appearance_type_id) REFERENCES appearance_type(appearance_type_id)
)
       ;
    """
# run the query
run_command(q)

# check that our table was created successfully
q3 = """
       SELECT * 
       FROM person_appearance
       LIMIT 10
       ;
    """
# run the query
run_query(q3)

,appearance_id,person_id,team_id,game_id,appearance_type_id


With our person_appearance table created, we'll now start inserting our data, starting with the following:
- umpires
  - home plate
  - first base
  - second base
  - third base
  - left field
  - right field
- pitchers
  - winning
  - losing
  - saving
  - starting
- winning RBI batter
- manager

In [61]:
# insert the data mentioned above
q = """ 

 INSERT OR IGNORE INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
)
    SELECT
        game_id,
        NULL,
        hp_umpire_id,
        "UHP"
    FROM game_log
    WHERE hp_umpire_id IS NOT NULL
    
UNION

    SELECT
        game_id,
        NULL,
        [1b_umpire_id],
        "U1B"
    FROM game_log
    WHERE [1b_umpire_id] IS NOT NULL
    
UNION

    SELECT
        game_id,
        NULL,
        [2b_umpire_id],
        "U2B"
    FROM game_log
    WHERE [2b_umpire_id] IS NOT NULL
    
    
UNION

    SELECT
        game_id,
        NULL,
        [3b_umpire_id],
        "U3B"
    FROM game_log
    WHERE [3b_umpire_id] IS NOT NULL
    
UNION
    
    SELECT
        game_id,
        NULL,
        lf_umpire_id,
        "ULF"
    FROM game_log
    WHERE lf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        rf_umpire_id,
        "URF"
    FROM game_log
    WHERE rf_umpire_id IS NOT NULL


UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_pitcher_id,
        "AWP"
    FROM game_log
    WHERE winning_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score < v_score THEN h_name
            ELSE v_name
            END,
        losing_pitcher_id,
        "ALP"
    FROM game_log
    WHERE losing_pitcher_id IS NOT NULL
    
UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        saving_pitcher_id,
        "ASP"
    FROM game_log
    WHERE saving_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_rbi_batter_id,
        "AWB"
    FROM game_log
    WHERE winning_rbi_batter_id IS NOT NULL

    
UNION

    SELECT
        game_id,
        v_name,
        v_manager_id,
        "MM"
    FROM game_log
    WHERE v_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_manager_id,
        "MM"
    FROM game_log
    WHERE h_manager_id IS NOT NULL
;
    """

# run the query
run_command(q)

# check that our table has data
q3 = """
       SELECT * 
       FROM person_appearance
       LIMIT 10
       ;
    """
# run the query
run_query(q3)

,appearance_id,person_id,team_id,game_id,appearance_type_id
0,1,maplb901,None,ALT188404300,UHP
1,2,curte801,ALT,ALT188404300,MM
2,3,sullt101,SLU,ALT188404300,MM
3,4,hoopm101,None,ALT188405020,UHP
4,5,curte801,ALT,ALT188405020,MM
5,6,sullt101,SLU,ALT188405020,MM
6,7,hoopm101,None,ALT188405030,UHP
7,8,curte801,ALT,ALT188405030,MM
8,9,sullt101,SLU,ALT188405030,MM
9,10,hoopm101,None,ALT188405050,UHP


With the above positions' data inserted, now let's insert the defensive and offensive positions. As a note, we'll use a for loop to go through and insert this data.

In [63]:
# Template query with the parameters that will be replaced by our for loop
template = """
INSERT INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "O{num}"
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL

UNION

    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "D" || CAST({hv}_player_{num}_def_pos AS INT)
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL;
"""

# for loop used to run our query for the different defensive and offensive positions
for hv in ["h","v"]:
    for num in range(1,10):
        query_vars = {
            "hv": hv,
            "num": num
        }
        run_command(template.format(**query_vars))

In [66]:
# Check that a sample of records were inserted correctly
q3 = """
       SELECT * 
       FROM person_appearance
       WHERE appearance_type_id IN ('D1')
       LIMIT 10
       ;
    """
# run the query
run_query(q3)

,appearance_id,person_id,team_id,game_id,appearance_type_id
0,1450207,tovac101,MIN,MIN196809220,D1
1,1601471,mannj101,BL1,BL1187410130,D1
2,1601473,mannj101,BL1,BL1187410140,D1
3,1615383,mcdej101,BR1,BR1187205070,D1
4,1615385,mcdej101,BR1,BR1187205090,D1
5,1615387,mcdej101,BR1,BR1187205180,D1
6,1615389,mcdej101,BR1,BR1187207060,D1
7,1615391,orouu101,BR1,BR1187207090,D1
8,1615405,zettg101,BR1,BR1187210220,D1
9,1615443,bondt101,BR2,BR2187405050,D1


Now that we have our normalized tables, we can drop the non-normalized versions imported from our csv's since we no longer need those tables.

In [70]:
# Drop each of our de-normalized tables since we no longer need them anymore
q = """
       DROP TABLE IF EXISTS game_log
       ;
    """
# run the query
run_command(q)

q2 = """
       DROP TABLE IF EXISTS park_codes
       ;
    """
# run the query
run_command(q2)

q3 = """
       DROP TABLE IF EXISTS team_codes
       ;
    """
# run the query
run_command(q3)

q4 = """
       DROP TABLE IF EXISTS person_codes
       ;
    """
# run the query
run_command(q4)

q5 = """
       DROP TABLE IF EXISTS appearance_type_original
       ;
    """
# run the query
run_command(q5)

# show our tables to see what we have left
show_tables()

,name,type
0,person,table
1,park,table
2,league,table
3,appearance_type,table
4,team,table
5,game,table
6,team_appearance,table
7,person_appearance,table


# Conclusion

Thank you for reading along! In this entry, we've created our own normalized set of tables from a de-normalized dataset. Until next time!